# Convert TIP3P into TIP4P as a test for how to handle virtual sites

The TIP4P model of water is shown at http://www.sklogwiki.org/SklogWiki/index.php/TIP4P_model_of_water ; TIP3P at http://www.sklogwiki.org/SklogWiki/index.php/TIP3P_model_of_water; this notebook roughly draws on these.

Our initial work is to simply generate TIP4P from TIP3P. After that we will attempt to validate that we have correctly generated TIP4P by validating OUR TIP4P energies against those from a saved system.

## Do some setup and imports

In [15]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk import unit
from sys import stdout
import numpy as np
from mdtraj.formats.gro import load_gro
import parmed
from copy import deepcopy

In [16]:
# Example compute_potential function from John Chodera
def compute_potential(system, positions):
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    context.applyConstraints(1.0e-5) # applyConstraints() or computeVirtualSites() required for virtual site position computation
    potential = context.getState(getEnergy=True).getPotentialEnergy()
    del context, integrator
    return potential

## Load the TIP3P System and set it up

In [17]:
# Prep the original system with TIP3P
# Load PDB
pdb = PDBFile('spc216.pdb')
# Load normal OpenMM water forcefield
forcefield = ForceField( 'tip3p.xml')
# get periodic box vectors
gro = load_gro('spc216.gro')
pdb.topology.setPeriodicBoxVectors(gro.openmm_boxes(0))

system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=0.6*unit.nanometer, constraints=HBonds, rigidWater = True)

## Build a new System and Topology with extra points/virtual sites

All blocks in this section should probably be run together

### First do some index bookkeeping

In [36]:
# Store indices of which hydrogen atoms are connected to which oxygen atom (by index) for use in adding virtual sites 
oxygen_hydrogens = {}
for residue in pdb.topology.residues():
    
    oxygen_index = None
    hydrogen_indices = []
    for atom in residue.atoms():
        if atom.element.name=='oxygen':
            oxygen_index = atom.index
        elif atom.element.name=='hydrogen':
            hydrogen_indices.append(atom.index)
        oxygen_hydrogens[oxygen_index] = hydrogen_indices


### Build new System and Topology

In [42]:
# Build a new System and Topology where we add the virtual sites/extra points; we modify the charges on
# the existing atoms at the same time

newSystem = System()
newTopology = Topology()
positions = pdb.getPositions()
newTopology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
newAtoms = {}
#newExtraPoints = {}
kwargs = {} # Figure out what values here should be
kwargs['nonbondedMethod']=PME
kwargs['nonbondedCutoff']=0.6*unit.nanometer
oldindex_to_newindex = {}
newindex_to_oldindex = {}
vsite_indices_by_atom = {}
hydrogens_by_residue = {}
oxygens_by_residue = {}

boxVectors = newTopology.getPeriodicBoxVectors()
if boxVectors is not None:
        newSystem.setDefaultPeriodicBoxVectors(boxVectors[0], boxVectors[1], boxVectors[2])


# Most straightforward procedure, I think, is to NOT do as I've started to do below, and instead:
# (a) Create System and Topology containing all of the desired atoms/residues (basically this is what ParmEd assumes -- System already has particles: https://github.com/ParmEd/ParmEd/blob/master/parmed/structure.py#L2161-L2197)
# (b) go through and add virtual sites and exceptions
# (c) Go through and copy over remaining forces

# Retrieve the old NonbondedForce
for force in system.getForces():
    if isinstance(force, openmm.NonbondedForce):
        #print(force)
        break

# Let's start on (a), building the System/Topology; particles occur as nonbonded forces
newForce = openmm.NonbondedForce()
newSystem.addForce(newForce)
for chain in pdb.topology.chains():
    newChain = newTopology.addChain(chain.id)
    for residue in chain.residues():
        newResidue = newTopology.addResidue(residue.name, newChain, residue.id)
        hydrogens_by_residue[newResidue.index] = []
        for atom in residue.atoms():
            # Add an atom corresponding to this particle to the new Topology
            newAtoms[atom] = newTopology.addAtom(atom.name, atom.element, newResidue)
            oldindex_to_newindex[atom.index] = newAtoms[atom].index
            newindex_to_oldindex[newAtoms[atom].index] = atom.index

            # Get particle parameters which were used
            chg, sigma, epsilon = force.getParticleParameters(atom.index)

            # Add a particle with those parameters to the new System/force
            newForce.addParticle(chg, sigma, epsilon)
            newSystem.addParticle(atom.element.mass)
            
            # If it's an oxygen, trigger addition of a new particle which will be the virtual site
            if atom.element.name=='oxygen':
                # Add to system/force
                # See https://gist.github.com/anonymous/be8693af41fe070638a27a834cfcb7e9 for how to work with virtual sites
                virtual_site_index = newSystem.addParticle(0.0 * unit.amu) # mass can be zero
                nonbonded_site_index = newForce.addParticle(-1.04*unit.elementary_charge, 0.0, 0.0) 

                # Check that these indices match, otherwise we're in trouble...
                assert virtual_site_index == nonbonded_site_index
                
                # Add to Topology
                topology_atom = newTopology.addAtom('VS', None, newResidue)
                assert topology_atom.index == virtual_site_index
                
                # Store index of virtual site, indexed by index of this oxygen
                vsite_indices_by_atom[newAtoms[atom].index] = virtual_site_index
                
                # Store oxygen index
                oxygens_by_residue[newResidue.index] = newAtoms[atom].index
            
            elif atom.element.name=='hydrogen':
                hydrogens_by_residue[newResidue.index].append( newAtoms[atom].index )
            
            # Store conversion from old to new indices
            oldindex_to_newindex[atom.index] = newAtoms[atom].index


## Update parameters to convert into TIP4P

(This should be run after the above and not repeated)

In [43]:
#(b) Go through system and add virtual sites and exceptions
# Also modify parameters of TIP3P oxygens and hydrogens at the same time to make consistent with TIP4P

# Retrieve the NonbondedForce
for force in newSystem.getForces():
    if isinstance(force, openmm.NonbondedForce):
        break

# Loop over residues, making changes to atoms and adding virtual site parameters
for residue in oxygens_by_residue:
    # Modify oxygen atom
    oxygenatom = oxygens_by_residue[residue]
    chg, sigma, epsilon = force.getParticleParameters(oxygenatom)
    force.setParticleParameters(oxygenatom, 0.*unit.elementary_charge, 3.154*unit.angstrom, 0.6480*unit.kilojoule_per_mole) 
                
    # Modify hydrogen atoms
    for hydrogenatom in hydrogens_by_residue[residue]:
        # Set hydrogen charge to 0.52
        chg, sigma, epsilon = force.getParticleParameters(hydrogenatom)
        force.setParticleParameters(hydrogenatom, 0.52*unit.elementary_charge, sigma, epsilon)
        
    # Add virtual site
    # see https://github.com/open-forcefield-group/openforcefield/pull/67#issuecomment-339398603
    site = LocalCoordinatesSite([oxygenatom, hydrogens_by_residue[residue][0], hydrogens_by_residue[residue][1]],\
                                       [1, 0, 0], [-1, 0.5, 0.5],[0.5, -0.5, 0],\
                                       [-0.015, 0, 0]) #WARNING: LocalCoordinatesSite does not take units; uses nanometers
    virtual_site_index = vsite_indices_by_atom[oxygenatom]
    newSystem.setVirtualSite(virtual_site_index, site)
            
    # Add exceptions involving the virtual site
    force.addException( virtual_site_index, oxygenatom, 0.0, 0.0, 0.0) #No interactions
    force.addException( virtual_site_index, hydrogens_by_residue[residue][0], 0.0, 0.0, 0.0)
    force.addException( virtual_site_index, hydrogens_by_residue[residue][1], 0.0, 0.0, 0.0)
    
    # Add original exceptions
    force.addException( oxygenatom, hydrogens_by_residue[residue][0], 0.0, 0.0, 0.0)
    force.addException( oxygenatom, hydrogens_by_residue[residue][1], 0.0, 0.0, 0.0)
    force.addException( hydrogens_by_residue[residue][0], hydrogens_by_residue[residue][1], 0.0, 0.0, 0.0)
    
    # Add constraints
    newSystem.addConstraint( oxygenatom, hydrogens_by_residue[residue][0], 0.9572*unit.angstrom)
    newSystem.addConstraint( oxygenatom, hydrogens_by_residue[residue][1], 0.9572*unit.angstrom)
    # Angle constraint comes in via trig -- 104.52 degrees results in a particular distance
    r_HH = 2.*0.9572*unit.angstroms*np.sin(104.52/2.*np.pi/180.)
    newSystem.addConstraint( hydrogens_by_residue[residue][0], hydrogens_by_residue[residue][1], r_HH)
    
#Form will be:
#newTopology.constrainAtomPair( index1, index2, constraint_distance)
print(force.getNumExceptions())

1296


In [44]:
#(c) Go through system and copy over remaining forces, using appropriate atom indices
# I don't think this actually has to be done here, except possibly COMM remover, as there ARE no other forces

for force in system.getForces():
    if isinstance(force, openmm.NonbondedForce):
        continue
    elif isinstance(force, openmm.HarmonicBondForce):
        print("%s bonds..." % force.getNumBonds())
    elif isinstance(force, openmm.HarmonicAngleForce):
        print("There are %s angle forces" % force.getNumAngles())
    else:
        print(force)



0 bonds...
There are 0 angle forces
<simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x12564c6f0> >


## Generate new starting positions for this system

We copy over the appropriate old coordinates, and just make dummy coordinates for the virtual sites, which will get updated by computing the virtual sites below.

In [45]:
# Generate positions appropriate for new system
newPositions = []*unit.nanometer
for atomidx in range(newSystem.getNumParticles()):
    if atomidx in newindex_to_oldindex:
        newPositions.append( pdb.positions[newindex_to_oldindex[atomidx]])
    else:  newPositions.append( [0.,0.,0.]*unit.angstrom)

## Energy minimize and dump out coordinates
We use a reporter to dump to `water_out.pdb`; it should end up with two frames, the first of which is the initial coordinates and the second is the final coordinates after minimization.

I inspected via PyMol and this appears reasonable.

In [46]:
# Try minimizing

integrator = openmm.VerletIntegrator(2.0*unit.femtoseconds)
simulation = app.Simulation(newTopology, newSystem, integrator) 
simulation.context.setPositions( newPositions) # Use new positions created above
#simulation.context.applyConstraints(1.0e-5) # applyConstraints() or computeVirtualSites() required for virtual site position computation
simulation.context.computeVirtualSites()

pdb_reporter = PDBReporter('water_out.pdb', 1)
simulation.reporters.append(pdb_reporter)
state = simulation.context.getState(getEnergy=True, getPositions=True)
pdb_reporter.report(simulation, state)
updated_positions = state.getPositions()

# Print info about distance from first oxygen atom to its virtual site
oxygen_index = oxygens_by_residue[0]
virtual_site_index = vsite_indices_by_atom[oxygens_by_residue[0]]
oxygen_pos = updated_positions[oxygen_index]
vsite_pos = updated_positions[virtual_site_index]
print("Distance from oxygen to virtual site: ", np.sqrt(np.dot(oxygen_pos-vsite_pos, oxygen_pos-vsite_pos)))


energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
print("Energy before minimization (kcal/mol): %.8g" % energy)

# Minimize energy, printing energy before and after
simulation.minimizeEnergy()
state = simulation.context.getState(getEnergy=True, getPositions=True)
energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
print("Energy after minimization (kcal/mol): %.8g" % energy)
newpositions = state.getPositions()
pdb_reporter.report(simulation, state)

Distance from oxygen to virtual site:  0.014999999996318784 nm
Energy before minimization (kcal/mol): -4343.1721
Energy after minimization (kcal/mol): -6673.7534


## Next steps from here

Now that the above appears to be working, logical next steps would be to:
- Make sure new system uses periodic boundary conditions/PME/constraints/rigid water
- Try short dynamics to make sure stable
- Cross-check energies against a normal TIP4P topology
  - atom order here will be different so we have to be careful in coordinate swapping
  - basically set up this system and a "standard" OpenMM TIP4P system; use equivalent positions and a single point energy evaluation to ensure they yield the same energy